In [2]:
import numpy as np
import pandas as pd
import os

In [4]:
DATA_EXTENSION = ".ssv"
ELEMENT_NAMES_EXTENSION = ".elems"


def get_raw_element_names(sample_name: str) -> list[str]:
    with open(f"{sample_name}/{sample_name}{ELEMENT_NAMES_EXTENSION}", "r", encoding="utf-8") as f:
        return f.read().split(",")


def convert_raw_element_names_to_sv12(raw_element_names: list[str]) -> tuple[list[str]]:
    sv1 = [f"{name}_sv1" for name in raw_element_names]
    sv2 = [f"{name}_sv2" for name in raw_element_names]
    return (sv1, sv2,)


def generate_dtype(
        element_names_sv1: list[str],
        element_names_sv2: list[str]
        ) -> list[tuple[str]]:
    base_dt_types = [
        ("scheme_state_sv1", "bool"),
        ("scheme_state_sv2", "bool"),
        ("scheme_state", "bool"),
        ("probability", "float64")
    ]
    dt_types_sv1 = [(name, "bool") for name in element_names_sv1]
    dt_types_sv2 = [(name, "bool") for name in element_names_sv2]
    return [*base_dt_types, *dt_types_sv1, *dt_types_sv2]


def read_directory_as_dataframe(sample_name: str, dtype) -> pd.DataFrame:
    return pd.concat(
        map(
            lambda file: pd.DataFrame(np.fromfile(os.path.join(sample_name, file), dtype=dtype)),
            filter(
                lambda file: file.endswith(DATA_EXTENSION),
                os.listdir(sample_name)
            )
        )
    )


def sum_probabilities(sample: pd.DataFrame) -> None:
    p = sample.loc[sample["scheme_state"] == True, "probability"].sum()
    q = sample.loc[sample["scheme_state"] == False, "probability"].sum()
    print(f"p = {p}")
    print(f"q = {q}")
    print(f"p + q = {p + q}")


def is_reconfiguration_same_or_better(sample: pd.DataFrame) -> bool:
    inconsistency = sample.loc[(sample["scheme_state_sv1"] == True) & (sample["scheme_state_sv2"] == False)]
    return inconsistency.empty


def sum_fail_probability_per_element(
        sample: pd.DataFrame,
        raw_element_names: list[str],
        element_names_sv1: list[str],
        element_names_sv2: list[str]
        ) -> None:
    failed_samples = sample.loc[sample["scheme_state"] == False]
    mask_sv1 = (failed_samples[element_names_sv1] == False)
    mask_sv2 = (failed_samples[element_names_sv2] == False)
    probabilities = failed_samples["probability"].values
    fail_probabilities_sv1 = np.sum(mask_sv1.values * probabilities[:, None], axis=0).tolist()
    fail_probabilities_sv2 = np.sum(mask_sv2.values * probabilities[:, None], axis=0).tolist()
    for i in range(len(raw_element_names)):
        print(f"{raw_element_names[i]}: sv1 = {fail_probabilities_sv1[i]:.8e}, sv2 = {fail_probabilities_sv2[i]:.8e}")


def sum_fail_count_per_element(
        sample: pd.DataFrame,
        raw_element_names: list[str],
        element_names_sv1: list[str],
        element_names_sv2: list[str]
        ) -> None:
    failed_samples_sv1 = sample.loc[sample["scheme_state"] == False]
    fail_count_sv1 = (failed_samples_sv1[element_names_sv1] == False).sum()
    failed_samples_sv2 = sample.loc[sample["scheme_state"] == False]
    fail_count_sv2 = (failed_samples_sv2[element_names_sv2] == False).sum()
    for i in range(len(raw_element_names)):
        print(f"{raw_element_names[i]}: sv1 = {fail_count_sv1.iloc[i]}, sv2 = {fail_count_sv2.iloc[i]}")


def process_sample(sample_name: str) -> tuple[list[str], list[str], list[str], pd.DataFrame]:
    raw_element_names = get_raw_element_names(sample_name)
    element_names_sv1, element_names_sv2 = convert_raw_element_names_to_sv12(raw_element_names)
    dtype = generate_dtype(element_names_sv1, element_names_sv2)
    
    samples = read_directory_as_dataframe(sample_name, dtype)

    print("=== sum probabilities ===")
    sum_probabilities(samples)

    if is_reconfiguration_same_or_better(samples):
        print("=== reconfiguration is consistent ===")
    else:
        print("=== reconfiguration is inconsistent ===")
        return

    print("=== sum fail probability(sv1 | sv2) per element ===")
    sum_fail_probability_per_element(
        samples,
        raw_element_names,
        element_names_sv1,
        element_names_sv2
    )

    print("=== sum fail count(sv1 | sv2) per element ===")
    sum_fail_count_per_element(
        samples,
        raw_element_names,
        element_names_sv1,
        element_names_sv2
    )

    return (raw_element_names, element_names_sv1, element_names_sv2, samples)

In [76]:
def simple():
    raw_element_names, element_names_sv1, element_names_sv2, samples = process_sample("simple")
#simple()

In [77]:
def process_sample_with_fail_count_per_123(sample_name: str) -> None:
    raw_element_names, element_names_sv1, element_names_sv2, samples = process_sample(sample_name)

    pr_cases_sv1 = (
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == False)],
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == True) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == False)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == True) & (samples["pr3_sv1"] == False)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == True)],
        samples.loc[(samples["pr1_sv1"] == False) & (samples["pr2_sv1"] == False) & (samples["pr3_sv1"] == False)]
    )

    pr_cases_sv2 = (
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == False)],
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == True) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == False)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == True) & (samples["pr3_sv2"] == False)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == True)],
        samples.loc[(samples["pr1_sv2"] == False) & (samples["pr2_sv2"] == False) & (samples["pr3_sv2"] == False)]
    )

    print("=== sum fail count(sv1 | sv2) per pr1 * pr2 * pr3 state set ===")
    for i in range(8):
        print(f"case {i}: sv1 = {pr_cases_sv1[i].shape[0]}; sv2 = {pr_cases_sv2[i].shape[0]}")

In [78]:
process_sample_with_fail_count_per_123("s23-original-greedy")

=== sum probabilities ===
p = 0.9990208569798806
q = 0.0009791430201191258
p + q = 0.9999999999999998
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.46262060e-07, sv2 = 2.87982720e-08
pr2: sv1 = 1.46262060e-07, sv2 = 2.87982720e-08
pr3: sv1 = 1.46262060e-07, sv2 = 2.87982720e-08
pr4: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
pr5: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 1.46871453e-08, sv2 = 1.46871453e-08
b2: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b4: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b5: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
c1: sv1 = 5.69315417e-07, sv2 = 5.69315417e-07
c2: sv1 = 5.69315417e-07, sv2 = 5.69315417e-07
c4: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c5: sv1 = 5.69315219e-07, sv2 = 5.69315219e-07
c6: sv1 = 5.69315219e-07, sv2 = 5.69315219e-07
d1: sv1 = 2.15413552e-08, sv2 = 2.15413552e-08
d2: 

In [79]:
process_sample_with_fail_count_per_123("s23-original-brute")

=== sum probabilities ===
p = 0.9990208569798806
q = 0.0009791430201191258
p + q = 0.9999999999999998
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 1.46262060e-07, sv2 = 2.87982720e-08
pr2: sv1 = 1.46262060e-07, sv2 = 2.87982720e-08
pr3: sv1 = 1.46262060e-07, sv2 = 2.87982720e-08
pr4: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
pr5: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 1.46871453e-08, sv2 = 1.46871453e-08
b2: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b4: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b5: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
c1: sv1 = 5.69315417e-07, sv2 = 5.69315417e-07
c2: sv1 = 5.69315417e-07, sv2 = 5.69315417e-07
c4: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c5: sv1 = 5.69315219e-07, sv2 = 5.69315219e-07
c6: sv1 = 5.69315219e-07, sv2 = 5.69315219e-07
d1: sv1 = 2.15413552e-08, sv2 = 2.15413552e-08
d2: 

In [80]:
process_sample_with_fail_count_per_123("s23-77788-greedy")

=== sum probabilities ===
p = 0.9992607082975082
q = 0.0007392917024916271
p + q = 0.9999999999999999
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 8.87253609e-08, sv2 = 1.38217191e-11
pr2: sv1 = 8.87253609e-08, sv2 = 1.38217191e-11
pr3: sv1 = 8.87253609e-08, sv2 = 1.38217191e-11
pr4: sv1 = 8.87253609e-08, sv2 = 1.55488898e-11
pr5: sv1 = 8.87253609e-08, sv2 = 1.55488898e-11
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 1.10893755e-08, sv2 = 1.10893755e-08
b2: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b4: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b5: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
c1: sv1 = 4.71016679e-07, sv2 = 4.71016679e-07
c2: sv1 = 4.71016679e-07, sv2 = 4.71016679e-07
c4: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c5: sv1 = 4.71016481e-07, sv2 = 4.71016481e-07
c6: sv1 = 4.71016481e-07, sv2 = 4.71016481e-07
d1: sv1 = 1.62646263e-08, sv2 = 1.62646263e-08
d2: 

In [81]:
process_sample_with_fail_count_per_123("s23-77788-brute")

=== sum probabilities ===
p = 0.9992607082975082
q = 0.0007392917024916271
p + q = 0.9999999999999999
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 8.87253609e-08, sv2 = 1.72800000e-12
pr2: sv1 = 8.87253609e-08, sv2 = 8.63875589e-12
pr3: sv1 = 8.87253609e-08, sv2 = 5.18337794e-12
pr4: sv1 = 8.87253609e-08, sv2 = 6.22030234e-16
pr5: sv1 = 8.87253609e-08, sv2 = 5.18337794e-12
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 1.10893755e-08, sv2 = 1.10893755e-08
b2: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b4: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
b5: sv1 = 1.50000000e-05, sv2 = 1.50000000e-05
c1: sv1 = 4.71016679e-07, sv2 = 4.71016679e-07
c2: sv1 = 4.71016679e-07, sv2 = 4.71016679e-07
c4: sv1 = 4.10000000e-04, sv2 = 4.10000000e-04
c5: sv1 = 4.71016481e-07, sv2 = 4.71016481e-07
c6: sv1 = 4.71016481e-07, sv2 = 4.71016481e-07
d1: sv1 = 1.62646263e-08, sv2 = 1.62646263e-08
d2: 

In [82]:
_, _, _, _ = process_sample("s23-77788-modified-connections-greedy")

=== sum probabilities ===
p = 0.9997377215216388
q = 0.0002622784783607355
p + q = 0.9999999999999994
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 3.14837790e-08, sv2 = 1.38217191e-11
pr2: sv1 = 3.14837790e-08, sv2 = 1.38217191e-11
pr3: sv1 = 3.14837790e-08, sv2 = 1.38217191e-11
pr4: sv1 = 3.14837790e-08, sv2 = 1.55488898e-11
pr5: sv1 = 3.14837790e-08, sv2 = 1.55488898e-11
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 4.15911479e-09, sv2 = 4.15911479e-09
b2: sv1 = 4.15911479e-09, sv2 = 4.15911479e-09
b4: sv1 = 4.15911479e-09, sv2 = 4.15911479e-09
b5: sv1 = 4.15911479e-09, sv2 = 4.15911479e-09
c1: sv1 = 1.07603051e-07, sv2 = 1.07603051e-07
c2: sv1 = 1.07603051e-07, sv2 = 1.07603051e-07
c4: sv1 = 1.07603051e-07, sv2 = 1.07603051e-07
c5: sv1 = 2.75521212e-07, sv2 = 2.75521212e-07
c6: sv1 = 2.75521212e-07, sv2 = 2.75521212e-07
d1: sv1 = 5.77012652e-09, sv2 = 5.77012652e-09
d2: 

In [83]:
_, _, _, _ = process_sample("s23-77788-modified-connections-brute")

=== sum probabilities ===
p = 0.9997377215216388
q = 0.0002622784783607355
p + q = 0.9999999999999994
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 3.14837790e-08, sv2 = 1.72800000e-12
pr2: sv1 = 3.14837790e-08, sv2 = 8.63875589e-12
pr3: sv1 = 3.14837790e-08, sv2 = 5.18337794e-12
pr4: sv1 = 3.14837790e-08, sv2 = 6.22030234e-16
pr5: sv1 = 3.14837790e-08, sv2 = 5.18337794e-12
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 4.15911479e-09, sv2 = 4.15911479e-09
b2: sv1 = 4.15911479e-09, sv2 = 4.15911479e-09
b4: sv1 = 4.15911479e-09, sv2 = 4.15911479e-09
b5: sv1 = 4.15911479e-09, sv2 = 4.15911479e-09
c1: sv1 = 1.07603051e-07, sv2 = 1.07603051e-07
c2: sv1 = 1.07603051e-07, sv2 = 1.07603051e-07
c4: sv1 = 1.07603051e-07, sv2 = 1.07603051e-07
c5: sv1 = 2.75521212e-07, sv2 = 2.75521212e-07
c6: sv1 = 2.75521212e-07, sv2 = 2.75521212e-07
d1: sv1 = 5.77012652e-09, sv2 = 5.77012652e-09
d2: 

In [84]:
_, _, _, _ = process_sample("s24-d9-right-greedy")

=== sum probabilities ===
p = 0.9997597157515121
q = 0.00024028424848726012
p + q = 0.9999999999999993
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 2.88444716e-08, sv2 = 1.38217191e-11
pr2: sv1 = 2.88444716e-08, sv2 = 1.38217191e-11
pr3: sv1 = 2.88444716e-08, sv2 = 1.38217191e-11
pr4: sv1 = 2.88444716e-08, sv2 = 1.55488898e-11
pr5: sv1 = 2.88444716e-08, sv2 = 1.55488898e-11
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 3.82920629e-09, sv2 = 3.82920629e-09
b2: sv1 = 3.82920629e-09, sv2 = 3.82920629e-09
b4: sv1 = 3.82920629e-09, sv2 = 3.82920629e-09
b5: sv1 = 3.82920629e-09, sv2 = 3.82920629e-09
c1: sv1 = 9.85854181e-08, sv2 = 9.85854181e-08
c2: sv1 = 9.85854181e-08, sv2 = 9.85854181e-08
c4: sv1 = 9.85854181e-08, sv2 = 9.85854181e-08
c5: sv1 = 2.66507274e-07, sv2 = 2.66507274e-07
c6: sv1 = 2.66507274e-07, sv2 = 2.66507274e-07
d1: sv1 = 5.28625347e-09, sv2 = 5.28625347e-09
d2:

In [85]:
_, _, _, _ = process_sample("s24-d9-right-brute")

=== sum probabilities ===
p = 0.9997597157515121
q = 0.00024028424848726012
p + q = 0.9999999999999993
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 2.88444716e-08, sv2 = 1.72800000e-12
pr2: sv1 = 2.88444716e-08, sv2 = 8.63875589e-12
pr3: sv1 = 2.88444716e-08, sv2 = 5.18337794e-12
pr4: sv1 = 2.88444716e-08, sv2 = 6.22030234e-16
pr5: sv1 = 2.88444716e-08, sv2 = 5.18337794e-12
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 3.82920629e-09, sv2 = 3.82920629e-09
b2: sv1 = 3.82920629e-09, sv2 = 3.82920629e-09
b4: sv1 = 3.82920629e-09, sv2 = 3.82920629e-09
b5: sv1 = 3.82920629e-09, sv2 = 3.82920629e-09
c1: sv1 = 9.85854181e-08, sv2 = 9.85854181e-08
c2: sv1 = 9.85854181e-08, sv2 = 9.85854181e-08
c4: sv1 = 9.85854181e-08, sv2 = 9.85854181e-08
c5: sv1 = 2.66507274e-07, sv2 = 2.66507274e-07
c6: sv1 = 2.66507274e-07, sv2 = 2.66507274e-07
d1: sv1 = 5.28625347e-09, sv2 = 5.28625347e-09
d2:

In [86]:
_, _, _, _ = process_sample("s25-d9-d10-right-greedy")

=== sum probabilities ===
p = 0.9997597162353858
q = 0.00024028376461420102
p + q = 1.0
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 2.88444135e-08, sv2 = 1.38217191e-11
pr2: sv1 = 2.88444135e-08, sv2 = 1.38217191e-11
pr3: sv1 = 2.88444135e-08, sv2 = 1.38217191e-11
pr4: sv1 = 2.88444135e-08, sv2 = 1.55488898e-11
pr5: sv1 = 2.88444135e-08, sv2 = 1.55488898e-11
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 3.82919903e-09, sv2 = 3.82919903e-09
b2: sv1 = 3.82919903e-09, sv2 = 3.82919903e-09
b4: sv1 = 3.82919903e-09, sv2 = 3.82919903e-09
b5: sv1 = 3.82919903e-09, sv2 = 3.82919903e-09
c1: sv1 = 9.85852197e-08, sv2 = 9.85852197e-08
c2: sv1 = 9.85852197e-08, sv2 = 9.85852197e-08
c4: sv1 = 9.85852197e-08, sv2 = 9.85852197e-08
c5: sv1 = 2.66507076e-07, sv2 = 2.66507076e-07
c6: sv1 = 2.66507076e-07, sv2 = 2.66507076e-07
d1: sv1 = 5.28624282e-09, sv2 = 5.28624282e-09
d2: sv1 = 5.286242

In [87]:
_, _, _, _ = process_sample("s25-d9-d10-right-brute")

=== sum probabilities ===
p = 0.9997597162353858
q = 0.00024028376461420102
p + q = 1.0
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 2.88444135e-08, sv2 = 1.72800000e-12
pr2: sv1 = 2.88444135e-08, sv2 = 8.63875589e-12
pr3: sv1 = 2.88444135e-08, sv2 = 5.18337794e-12
pr4: sv1 = 2.88444135e-08, sv2 = 6.22030234e-16
pr5: sv1 = 2.88444135e-08, sv2 = 5.18337794e-12
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 3.82919903e-09, sv2 = 3.82919903e-09
b2: sv1 = 3.82919903e-09, sv2 = 3.82919903e-09
b4: sv1 = 3.82919903e-09, sv2 = 3.82919903e-09
b5: sv1 = 3.82919903e-09, sv2 = 3.82919903e-09
c1: sv1 = 9.85852197e-08, sv2 = 9.85852197e-08
c2: sv1 = 9.85852197e-08, sv2 = 9.85852197e-08
c4: sv1 = 9.85852197e-08, sv2 = 9.85852197e-08
c5: sv1 = 2.66507076e-07, sv2 = 2.66507076e-07
c6: sv1 = 2.66507076e-07, sv2 = 2.66507076e-07
d1: sv1 = 5.28624282e-09, sv2 = 5.28624282e-09
d2: sv1 = 5.286242

In [5]:
_, _, _, _ = process_sample("s27-d9-d10-right-c8-left-greedy")

=== sum probabilities ===
p = 0.9997598842949944
q = 0.00024011570500589912
p + q = 1.0000000000000002
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 2.88242464e-08, sv2 = 1.38217191e-11
pr2: sv1 = 2.88242464e-08, sv2 = 1.38217191e-11
pr3: sv1 = 2.88242464e-08, sv2 = 1.38217191e-11
pr4: sv1 = 2.88242464e-08, sv2 = 1.55488898e-11
pr5: sv1 = 2.88242464e-08, sv2 = 1.55488898e-11
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 3.82667817e-09, sv2 = 3.82667817e-09
b2: sv1 = 3.82667817e-09, sv2 = 3.82667817e-09
b4: sv1 = 3.82667817e-09, sv2 = 3.82667817e-09
b5: sv1 = 3.82667817e-09, sv2 = 3.82667817e-09
c1: sv1 = 9.84474673e-08, sv2 = 9.84474673e-08
c2: sv1 = 9.84474673e-08, sv2 = 9.84474673e-08
c4: sv1 = 9.84474673e-08, sv2 = 9.84474673e-08
c5: sv1 = 9.85163153e-08, sv2 = 9.85163153e-08
c6: sv1 = 9.85163153e-08, sv2 = 9.85163153e-08
d1: sv1 = 5.28254551e-09, sv2 = 5.28254551e-09
d2:

In [6]:
_, _, _, _ = process_sample("s27-d9-d10-right-c8-left-brute")

=== sum probabilities ===
p = 0.9997598842949944
q = 0.00024011570500589912
p + q = 1.0000000000000002
=== reconfiguration is consistent ===
=== sum fail probability(sv1 | sv2) per element ===
pr1: sv1 = 2.88242464e-08, sv2 = 1.72800000e-12
pr2: sv1 = 2.88242464e-08, sv2 = 8.63875589e-12
pr3: sv1 = 2.88242464e-08, sv2 = 5.18337794e-12
pr4: sv1 = 2.88242464e-08, sv2 = 6.22030234e-16
pr5: sv1 = 2.88242464e-08, sv2 = 5.18337794e-12
a1: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
a2: sv1 = 1.20000000e-04, sv2 = 1.20000000e-04
b1: sv1 = 3.82667817e-09, sv2 = 3.82667817e-09
b2: sv1 = 3.82667817e-09, sv2 = 3.82667817e-09
b4: sv1 = 3.82667817e-09, sv2 = 3.82667817e-09
b5: sv1 = 3.82667817e-09, sv2 = 3.82667817e-09
c1: sv1 = 9.84474673e-08, sv2 = 9.84474673e-08
c2: sv1 = 9.84474673e-08, sv2 = 9.84474673e-08
c4: sv1 = 9.84474673e-08, sv2 = 9.84474673e-08
c5: sv1 = 9.85163153e-08, sv2 = 9.85163153e-08
c6: sv1 = 9.85163153e-08, sv2 = 9.85163153e-08
d1: sv1 = 5.28254551e-09, sv2 = 5.28254551e-09
d2: